In [1]:
import wrds
import pandas as pd

In [2]:
# data = pd.read_csv('signed_predictors_all_wide.csv')
# data.to_parquet('dataset.parquet')

data = pd.read_parquet('dataset.parquet') # TAKES LESS TIME TO LOAD BUT STILL ~1 / 2  MIN

In [4]:
db = wrds.Connection()  # It will ask you your username and password

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [5]:
# get all hcomnam, hsiccd codes for each permno
hsiccd = db.raw_sql("select PERMNO, hcomnam, hsiccd from crsp.dsfhdr")
# change hsiccd type to int
hsiccd['hsiccd'] = hsiccd['hsiccd'].astype(int)
# new data frame of all unique permno
permno = pd.DataFrame(data['permno'].unique(), columns=['permno'])
# merge the sic codes with the permno
permno_hsiccd = permno.merge(hsiccd, on='permno')
permno_hsiccd['industry'] = permno_hsiccd['hsiccd'].apply(lambda x: x // 10)
permno_hsiccd['major_group'] = permno_hsiccd['hsiccd'].apply(lambda x: x // 100)
permno_hsiccd.rename(columns={'hcomnam': 'company_name', 'hsiccd': 'sic'}, inplace=True)

In [6]:
# Load each CSV file into a separate DataFrame
df4 = pd.read_csv('sic_tables/divisions.csv')
df3 = pd.read_csv('sic_tables/major-groups.csv')
df2 = pd.read_csv('sic_tables/industry-groups.csv')
df1 = pd.read_csv('sic_tables/sic-codes.csv')

# Rename the 'Description' column in each DataFrame
df4.rename(columns={'Description': 'Division_Description'}, inplace=True)
df3.rename(columns={'Description': 'Major_Description'}, inplace=True)
df2.rename(columns={'Description': 'Industry_Description'}, inplace=True)
df1.rename(columns={'Description': 'SIC_Description'}, inplace=True)

# Merge the DataFrames
# Merge permno_hsiccd with df1 on the 'sic' column
merged_df = permno_hsiccd.merge(df1, left_on='sic', right_on='SIC', how='left')
merged_df = merged_df[['permno', 'company_name', 'industry', 'major_group', 'SIC_Description']]
# Merge permno_hsiccd with df2 on the 'Industry Group' column
merged_df = merged_df.merge(df2, left_on='industry', right_on='Industry Group', how='left')
merged_df = merged_df[['permno', 'company_name', 'Industry_Description', 'major_group', 'SIC_Description']]

# Merge permno_hsiccd with df3 and df4 on the 'Major Groups' column
merged_df = merged_df.merge(df3, left_on='major_group', right_on='Major Group', how='left')
merged_df = merged_df[
    ['permno', 'company_name', 'Major_Description', 'Industry_Description', 'SIC_Description', 'Division']]

merged_df = merged_df.merge(df4, on='Division', how='left')
merged_df = merged_df[
    ['permno', 'company_name', 'Division_Description', 'Major_Description', 'Industry_Description',
     'SIC_Description']]

In [7]:
merged_df.head()

,permno,company_name,Division_Description,Major_Description,Industry_Description,SIC_Description
0,10000,OPTIMUM MANUFACTURING INC,Manufacturing,Miscellaneous Manufacturing Industries,Miscellaneous Manufacturing Industries,NaN
1,10001,GAS NATURAL INC,"Transportation, Communications, Electric, Gas,...","Electric, Gas, And Sanitary Services",Gas Production And Distribution,"Mixed, Manufactured, or Liquefied Petroleum Ga..."
2,10002,BANCTRUST FINANCIAL GROUP INC,"Finance, Insurance, And Real Estate",Depository Institutions,Commercial Banks,NaN
3,10003,GREAT COUNTRY BK ASONIA CT,"Finance, Insurance, And Real Estate",Depository Institutions,Commercial Banks,NaN
4,10004,CLOSE OUTS PLUS INC,Retail Trade,General Merchandise Stores,Variety Stores,NaN


In [8]:
merged_df.isna().sum()

permno                      0
company_name                0
Division_Description      335
Major_Description         335
Industry_Description     1224
SIC_Description         12048
dtype: int64

In [ ]:
merged_df.to_csv('permno_full_info1.csv', index=False)